In [1]:
import email
import os
import re
import hashlib
import quopri

In [2]:
INPUT_DIR = '/Users/worshamn/Documents/emailProject/Investigate'
OUTPUT_DIR = '/Users/worshamn/Documents/emailProject/InvestigateFix'

In [3]:
def find_message_attachment(mail):
    if mail.is_multipart():
        for i in mail.get_payload():
            try:
                if i.get_content_maintype() == 'message':
                    return i.get_payload()[0]
            except:
                continue

In [4]:
#https://github.com/ActiveState/code/blob/master/recipes/Python/302086_Strip_attachments_email/recipe-302086.py
#https://github.com/xme/mime2vt/blob/master/mime2vt.py
#https://stackoverflow.com/a/370038
def strip_attachments(message):
    if not message.is_multipart():
        reformatted_message = quopri.decodestring(
                                message.as_string().encode('ascii', 'ignore')
                            ).decode("utf-8",'ignore')
        return reformatted_message
    boundary = message.get_boundary()
    new_payload = '--' + boundary
    for i in message.get_payload():
        if i.get_content_maintype() == 'message':
            return strip_attachements(i)
        elif i.get_content_maintype() == 'text' \
                or i.get_content_maintype() == 'multipart':
            text_content = i.as_string().encode('ascii', 'ignore')
            text_content = quopri.decodestring(text_content).decode("utf-8",'ignore')
            new_payload += '\n' + text_content
        else:
            replace = re.sub(r'(?:\n\n)[\s\S]+',r'\n\nREMOVED ATTACHMENT',i.as_string())
            content_type = i.get_content_type()
            filename = i.get_filename()
            charset = i.get_content_charset()
            md5 = hashlib.md5(i.get_payload(None,True)).hexdigest()
            sha256 = hashlib.sha256(i.get_payload(None,True)).hexdigest()
            replace_string = """
content_type=%(content_type)s
filename=%(filename)s
charset=%(charset)s
md5=%(md5)s
sha256=%(sha256)s
"""
            metadata = replace_string % dict(
                content_type=content_type, 
                filename=filename, 
                charset=charset,
                md5=md5,
                sha256=sha256,
            )
            new_payload += '\n' \
                + replace \
                + metadata
        new_payload += '\n--' + boundary
    new_payload += '--'
    message.set_payload(new_payload)
    return message.as_string()

In [5]:
def clean_move(input_dir=INPUT_DIR, output_dir=OUTPUT_DIR):
    for i in os.listdir(input_dir):
            f = os.path.join(input_dir, i)
            with open(f,'r',encoding='utf-8', errors='ignore') as raw_mail:
                mail = email.message_from_file(raw_mail)
                mail = find_message_attachment(mail)
                if mail:
                    mail = strip_attachments(mail)
                    with open(os.path.join(output_dir, i),'w') as out:
                        out.write(mail)
                else:
                    print(i)        

In [6]:
clean_move()

FW- Remittance #49170.eml
[Phish Alert] FW- Compass review.eml
[Phish Alert] FW- Thursday[3].eml
[Phish Alert] FW- Mark D. Goodman Sent you an important document..[1].eml
[Phish Alert] FW- Important document.eml
[Phish Alert] FW- Compass review[2].eml
[Phish Alert] FW- Mark D. Goodman Sent you an important document...eml
.DS_Store
[Phish Alert] FW- SystemForward America, Inc..eml
[Phish Alert] FW- Capitoline Consulting, LLC.eml
[Phish Alert] FW- Transfer Done[1].eml
[Phish Alert] FW- Thursday[2].eml
[Phish Alert] FW- Report[1].eml
[Phish Alert] FW- Thursday[9].eml
[Phish Alert] FW- Patty.Satanton shared a document with you via Dropbox.eml
[Phish Alert] FW- C2c-443655[1].eml
[Phish Alert] FW- PDF DOC.eml
[Phish Alert] FW- Thursday.eml
[Phish Alert] FW- Thursday[5].eml
[Phish Alert] FW- Thursday[4].eml
[Phish Alert] FW- YOUR IMMEDIATE ACTION IS REQUIRED.eml
[Phish Alert] FW- PO#6776.eml
[Phish Alert] FW- See file.eml
[Phish Alert] FW- Please Check.eml
[Phish Alert] FW- Thursday[8].eml
Fw

In [7]:
len(os.listdir(INPUT_DIR))

534

In [8]:
len(os.listdir(OUTPUT_DIR))

480